In [12]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
import lda


In [13]:
app = pd.read_pickle('app_cleaned.pickle')

In [14]:
app.columns

Index([u'category', u'current_rating', u'description', u'id',
       u'is_InAppPurcased', u'is_multilingual', u'is_multiplatform', u'name',
       u'new_version_desc', u'num_current_rating', u'num_overall_rating',
       u'overall_rating', u'price', u'publish_date', u'review1',
       u'review1_star', u'review2', u'review2_star', u'review3',
       u'review3_star', u'scrape_date', u'seller', u'size', u'update_date',
       u'url', u'version'],
      dtype='object')

In [ ]:
app['weighted_rating'] = map(lambda a, b, c,d: np.divide(a,b)*c+(1-np.divide(a,b))*d, app['num_current_rating'], 
                                  app['num_overall_rating'], app['current_rating'], app['overall_rating'])

Simply use the metric we created to define the quality of a app. If the weighted rating is no less than 4.0, it can be seen as a good app. If the weighted rating is no more than 2.5, it is a bad app.

In [18]:
good_app = app.loc[app['weighted_rating'] >=4.0]
bad_app = app.loc[app['weighted_rating'] <=2.5]
good_app = good_app.reset_index(drop=True)
bad_app = bad_app.reset_index(drop=True)


In [17]:
category = app['category']
cate_list = []
for i in category.unique():
    cate = i.lower()
    cate_list.append(cate)

Use star value of different reviews to filter comments. 

In [19]:
first_good= good_app.loc[good_app['review1_star']>=4].reset_index(drop=True)['review1']
second_good = good_app.loc[good_app['review2_star']>=4].reset_index(drop=True)['review2']
third_good = good_app.loc[good_app['review3_star']>=4].reset_index(drop=True)['review3']
first_bad = bad_app.loc[bad_app['review1_star']<=2.5].reset_index(drop=True)['review1']
second_bad = bad_app.loc[bad_app['review2_star']<=2.5].reset_index(drop=True)['review2']
third_bad = bad_app.loc[bad_app['review3_star']<=2.5].reset_index(drop=True)['review3']

In [20]:
good_rev = first_good.append(second_good)
all_good = good_rev.append(third_good)
bad_rev = first_bad.append(second_bad)
all_bad = bad_rev.append(third_bad)

<b>Cleaning and Preprocessing</b>

Data cleaning is absolutely crucial for generating a useful topic model. The steps below are common to most natural language processing methods:
   * Tokenizing: converting a document to its atomic elements.
   * Stopping: removing meaningless words.
   * Stemming: merging words that are equivalent in meaning.

Here we need to note that POS tag filter is more about the context of the features than frequencies of features. Topic Modelling tries to map out the recurring patterns of terms into topics. However, every term might not be equally important contextually. For example, POS tag IN contain terms such as – “within”, “upon”, “except”. “CD” contains – “one”,”two”, “hundred” etc. “MD” contains “may”, “must” etc. These terms are the supporting words of a language and can be removed by studying their post tags.

In [21]:
stop = set(stopwords.words('english')+[u'one',u'app',u'it',u'dont',u"i",u"'s","''","``",u'use',u'used',u'using',u'love',
                                      u'would',u'great',u'app.',u'like',u'lot']+ cate_list)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def stem(tokens,stemmer = PorterStemmer().stem):
    return [stemmer(w.lower()) for w in tokens if w not in stop]

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    tokenize = nltk.word_tokenize
    to_token = stem(tokenize(normalized))
    tags = nltk.pos_tag(to_token)
    dt_tags = [t for t in tags if t[1] in ["DT", "MD", "VBP","IN", "JJ","VB"]]
    for tag in dt_tags:
        normalized = " ".join(tok for tok in to_token if tok not in tag[0])
    return normalized

In [22]:
doc_clean_g1 = [clean(doc).split() for doc in first_good]
doc_clean_g2 = [clean(doc).split() for doc in second_good]
doc_clean_g3 = [clean(doc).split() for doc in third_good]
doc_clean_b1 = [clean(doc).split() for doc in first_bad]
doc_clean_b2 = [clean(doc).split() for doc in second_bad]
doc_clean_b3 = [clean(doc).split() for doc in third_bad]

In [23]:
doc_clean_good = [clean(doc).split() for doc in all_good]
doc_clean_bad = [clean(doc).split() for doc in all_bad]

<b>Preparing Document-Term Matrix</b>
* Convert a corpus into a document-term matrix. LDA model looks for repeating term patterns in the entire DT matrix

In [24]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary_g1 = corpora.Dictionary(doc_clean_g1)
dictionary_g2 = corpora.Dictionary(doc_clean_g2)
dictionary_g3 = corpora.Dictionary(doc_clean_g3)
dictionary_b1 = corpora.Dictionary(doc_clean_b1)
dictionary_b2 = corpora.Dictionary(doc_clean_b2)
dictionary_b3 = corpora.Dictionary(doc_clean_b3)

In [25]:
dictionary_good = corpora.Dictionary(doc_clean_good)
dictionary_bad = corpora.Dictionary(doc_clean_bad)

In [26]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix_g1 = [dictionary_g1.doc2bow(doc) for doc in doc_clean_g1]
doc_term_matrix_g2 = [dictionary_g2.doc2bow(doc) for doc in doc_clean_g2]
doc_term_matrix_g3 = [dictionary_g3.doc2bow(doc) for doc in doc_clean_g3]
doc_term_matrix_b1 = [dictionary_b1.doc2bow(doc) for doc in doc_clean_b1]
doc_term_matrix_b2 = [dictionary_b2.doc2bow(doc) for doc in doc_clean_b2]
doc_term_matrix_b3 = [dictionary_b3.doc2bow(doc) for doc in doc_clean_b3]

In [27]:
doc_term_matrix_good = [dictionary_good.doc2bow(doc) for doc in doc_clean_good]
doc_term_matrix_bad = [dictionary_bad.doc2bow(doc) for doc in doc_clean_bad]

<b>Running LDA Model (Batch Wise LDA)</b>
   * According to the [reference](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/), in order to retrieve most important topic terms, a corpus can be divided into batches of fixed sizes. Running LDA multiple times on these batches will provide different results, however, the best topic terms will be the intersection of all batches. 


In [28]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [29]:
# Running and Trainign LDA model on the document term matrix.
ldamodel_g1 = Lda(doc_term_matrix_g1, num_topics=3, id2word = dictionary_g1, passes=50)
ldamodel_g2 = Lda(doc_term_matrix_g2, num_topics=3, id2word = dictionary_g2, passes=50)
ldamodel_g3 = Lda(doc_term_matrix_g3, num_topics=3, id2word = dictionary_g3, passes=50)
ldamodel_b1 = Lda(doc_term_matrix_b1, num_topics=3, id2word = dictionary_b1, passes=50)
ldamodel_b2 = Lda(doc_term_matrix_b2, num_topics=3, id2word = dictionary_b2, passes=50)
ldamodel_b3 = Lda(doc_term_matrix_b3, num_topics=3, id2word = dictionary_b3, passes=50)

<b>Examining the results</b>

In [30]:
print(ldamodel_g1.print_topics(num_topics=3, num_words=5))
print(ldamodel_g2.print_topics(num_topics=3, num_words=5))
print(ldamodel_g3.print_topics(num_topics=3, num_words=5))

[(0, u'0.009*"get" + 0.007*"im" + 0.006*"help" + 0.006*"realli" + 0.006*"work"'), (1, u'0.014*"time" + 0.012*"game" + 0.009*"get" + 0.008*"play" + 0.006*"fun"'), (2, u'0.009*"easi" + 0.008*"get" + 0.008*"time" + 0.007*"make" + 0.007*"work"')]
[(0, u'0.012*"time" + 0.007*"also" + 0.006*"ive" + 0.005*"work" + 0.005*"go"'), (1, u'0.007*"time" + 0.007*"work" + 0.006*"app" + 0.005*"photo" + 0.004*"thank"'), (2, u'0.014*"get" + 0.009*"game" + 0.009*"time" + 0.008*"realli" + 0.008*"im"')]
[(0, u'0.008*"time" + 0.008*"get" + 0.008*"work" + 0.008*"see" + 0.007*"make"'), (1, u'0.013*"get" + 0.008*"help" + 0.006*"good" + 0.005*"well" + 0.005*"look"'), (2, u'0.012*"time" + 0.010*"ive" + 0.008*"best" + 0.007*"work" + 0.007*"make"')]


Each generated topic is separated by a comma. Within each topic are the five most probable words to appear in that topic. The best topic terms will be the intersection of all three batches. Some things to think about, for the good app, the comments have common features like: 
  1. It's free have some good features that satisfy customers' demand.
  2. It has many good information and details, and customers are comfortable at vision, like screen.
  3. The speed is awesome and save some time.
  4. It provids some help when customers using it.


In [31]:
print(ldamodel_b1.print_topics(num_topics=3, num_words=5))
print(ldamodel_b2.print_topics(num_topics=3, num_words=5))
print(ldamodel_b3.print_topics(num_topics=3, num_words=5))

[(0, u'0.007*"time" + 0.007*"peopl" + 0.006*"tri" + 0.005*"websit" + 0.005*"im"'), (1, u'0.012*"get" + 0.012*"time" + 0.008*"work" + 0.007*"want" + 0.007*"even"'), (2, u'0.012*"time" + 0.009*"work" + 0.007*"even" + 0.007*"get" + 0.007*"go"')]
[(0, u'0.015*"time" + 0.014*"work" + 0.010*"get" + 0.008*"tri" + 0.008*"make"'), (1, u'0.010*"get" + 0.010*"time" + 0.007*"work" + 0.006*"tri" + 0.006*"even"'), (2, u'0.009*"get" + 0.008*"time" + 0.007*"work" + 0.007*"tri" + 0.007*"updat"')]
[(0, u'0.015*"time" + 0.008*"tri" + 0.007*"work" + 0.006*"give" + 0.006*"back"'), (1, u'0.011*"time" + 0.009*"work" + 0.009*"get" + 0.007*"need" + 0.007*"even"'), (2, u'0.010*"work" + 0.008*"get" + 0.007*"time" + 0.006*"even" + 0.006*"download"')]


For the bad apps, from the result, we can see most topics include the word "time". We can refer that customers are not satisfied for the using fluency of these apps. And for the updated version of these apps, they doesn't work sometimes, maybe because flashing back. Meanwhile, compared with the last version, these updated apps maybe designed not that good. 


<b>Running LDA Model (For the whole documents)</b>

In [32]:
ldamodel_good = Lda(doc_term_matrix_good, num_topics=10, id2word = dictionary_good, passes=20)
ldamodel_bad = Lda(doc_term_matrix_bad, num_topics=10, id2word = dictionary_bad, passes=20)

In [32]:
print(ldamodel_good.print_topics(num_topics=5, num_words=3))
print(ldamodel_bad.print_topics(num_topics=5, num_words=3))

[(5, u'0.040*open + 0.029*see + 0.027*screen'), (2, u'0.013*time + 0.011*im + 0.011*get'), (9, u'0.030*accur + 0.027*also + 0.025*time'), (7, u'0.026*day + 0.020*everi + 0.014*help'), (3, u'0.021*inform + 0.020*locat + 0.020*map')]
[(6, u'0.013*time + 0.011*work + 0.010*even'), (9, u'0.014*work + 0.013*time + 0.009*doesnt'), (0, u'0.013*get + 0.012*work + 0.012*time'), (5, u'0.012*page + 0.011*time + 0.010*get'), (2, u'0.016*time + 0.009*need + 0.007*show')]


In [56]:
import pyLDAvis
import pyLDAvis.gensim

/Users/liwenyu/anaconda/envs/my_env/lib/python2.7/site-packages/simplejson/encoder.py:262: DeprecationWarning: Interpreting naive datetime as local 2017-03-22 18:42:34.119736. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
pyLDAvis.enable_notebook()
good_rev = pyLDAvis.gensim.prepare(ldamodel_good, doc_term_matrix_good, dictionary_good)
bad_rev = pyLDAvis.gensim.prepare(ldamodel_bad, doc_term_matrix_bad, dictionary_bad)

In [65]:
pyLDAvis.save_html(good_rev,"good_rev.html")
good_rev

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      15.088907        1       1  0.088825 -0.030739
0      15.069141        1       2  0.067187 -0.023132
2      13.849269        1       3  0.060865 -0.033778
6      13.434389        1       4  0.058241  0.025104
4      12.068342        1       5  0.016451  0.038235
9       8.616579        1       6  0.087835 -0.129834
1       7.336037        1       7  0.041543 -0.000540
5       7.213450        1       8  0.005044  0.033906
3       4.741021        1       9 -0.078040  0.213141
8       2.582865        1      10 -0.347952 -0.092364, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
4171  Default   648.000000       game   648.000000  30.0000  30.0000
1510  Default   441.000000       play   441.000000  29.0000  29.0000
6576  Default   378.000000        fun   378.000000  28.0000  28.0000
3562  Default   362.000000       read   362.000000  27.0000  27.0000
3845  Default   854.000000       help   854.000000  26.0000  26.0000
2970  Default   346.000000      photo   346.000000  25.0000  25.0000
8656  Default   217.000000       song   217.000000  24.0000  24.0000
7289  Default   304.000000     pictur   304.000000  23.0000  23.0000
5877  Default  1390.000000        get  1390.000000  22.0000  22.0000
5566  Default   237.000000       item   237.000000  21.0000  21.0000
4878  Default   343.000000       star   343.000000  20.0000  20.0000
2607  Default   214.000000       word   214.000000  19.0000  19.0000
1353  Default   246.000000      learn   246.000000  18.0000  18.0000
4651  Default   487.000000      thank   487.000000  17.0000  17.0000
6007  Default   247.000000      money   247.000000  16.0000  16.0000
726   Default   393.000000      peopl   393.000000  15.0000  15.0000
7367  Default   145.000000      sleep   145.000000  14.0000  14.0000
1357  Default    75.000000       scan    75.000000  13.0000  13.0000
8377  Default   202.000000      color   202.000000  12.0000  12.0000
4986  Default   487.000000  recommend   487.000000  11.0000  11.0000
1183  Default   404.000000         ad   404.000000  10.0000  10.0000
9026  Default    80.000000       bibl    80.000000   9.0000   9.0000
3523  Default    66.000000   document    66.000000   8.0000   8.0000
497   Default   811.000000       easi   811.000000   7.0000   7.0000
6635  Default   614.000000        app   614.000000   6.0000   6.0000
5057  Default   158.000000    workout   158.000000   5.0000   5.0000
376   Default   590.000000       best   590.000000   4.0000   4.0000
9826  Default   122.000000       appl   122.000000   3.0000   3.0000
7462  Default   526.000000      phone   526.000000   2.0000   2.0000
4983  Default   195.000000       call   195.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
5247  Topic10     4.922680       sail     5.803634   3.4916  -6.6765
7464  Topic10     4.788067  excellent     5.668959   3.4874  -6.7043
1752  Topic10    10.841522      sampl    12.842379   3.4869  -5.8870
2543  Topic10    32.570694     upload    48.440530   3.2593  -4.7870
9862  Topic10     8.745804       boom    11.365921   3.3942  -6.1018
6314  Topic10    19.102580      radar    31.986415   3.1408  -5.3206
5472  Topic10    22.636246     editor    43.075520   3.0129  -5.1508
3374  Topic10    16.228577    browser    27.569062   3.1264  -5.4836
4393  Topic10    29.972844       meet    69.721876   2.8120  -4.8701
3924  Topic10    11.341451     assign    17.648907   3.2141  -5.8419
9559  Topic10    27.697786      class    78.878427   2.6097  -4.9490
9419  Topic10     8.772112     attend    13.109268   3.2545  -6.0988
6370  Topic10    18.748383      print    49.039705   2.6947  -5.3393
7394  Topic10    23.993050     school    94.646761   2.2839  -5.0926
5129  Topic10    10.546621      grade    20.424467   2.9953  -5.91

/Users/liwenyu/anaconda/envs/my_env/lib/python2.7/site-packages/simplejson/encoder.py:262: DeprecationWarning: Interpreting naive datetime as local 2017-03-22 18:56:13.639523. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [63]:
bad_rev
pyLDAvis.save_html(bad_rev,"bad_rev.html")
bad_rev

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      21.452135        1       1  0.061510  0.014512
9      18.311871        1       2  0.036599  0.060238
1      10.221473        1       3  0.044358 -0.019721
0       8.374573        1       4  0.032911 -0.020142
7       8.211916        1       5 -0.037597  0.021870
5       7.192654        1       6  0.036634  0.006666
4       7.187352        1       7 -0.137567  0.085762
6       7.173991        1       8  0.042231  0.023743
8       6.812950        1       9 -0.013310 -0.029726
2       5.061084        1      10 -0.065768 -0.143201, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
1784  Default   40.000000        tv   40.000000  30.0000  30.0000
241   Default   51.000000      bank   51.000000  29.0000  29.0000
1852  Default   56.000000   version   56.000000  28.0000  28.0000
3193  Default   61.000000     email   61.000000  27.0000  27.0000
529   Default   36.000000     login   36.000000  26.0000  26.0000
1004  Default   63.000000     peopl   63.000000  25.0000  25.0000
2805  Default   52.000000    websit   52.000000  24.0000  24.0000
2295  Default   84.000000       see   84.000000  23.0000  23.0000
1582  Default   70.000000      read   70.000000  22.0000  22.0000
2393  Default   70.000000      look   70.000000  21.0000  21.0000
2094  Default   15.000000     print   15.000000  20.0000  20.0000
3464  Default   32.000000      appl   32.000000  19.0000  19.0000
1377  Default  224.000000       get  224.000000  18.0000  18.0000
3168  Default   42.000000      view   42.000000  17.0000  17.0000
1707  Default   56.000000   connect   56.000000  16.0000  16.0000
1509  Default   71.000000       pay   71.000000  15.0000  15.0000
145   Default  105.000000      want  105.000000  14.0000  14.0000
2789  Default   20.000000   restaur   20.000000  13.0000  13.0000
599   Default   24.000000    receiv   24.000000  12.0000  12.0000
1849  Default   57.000000       abl   57.000000  11.0000  11.0000
1143  Default   15.000000   magazin   15.000000  10.0000  10.0000
3245  Default   19.000000     creat   19.000000   9.0000   9.0000
2734  Default   80.000000  download   80.000000   8.0000   8.0000
1193  Default   37.000000      card   37.000000   7.0000   7.0000
3095  Default   24.000000   payment   24.000000   6.0000   6.0000
3380  Default   67.000000      page   67.000000   5.0000   5.0000
132   Default  214.000000      work  214.000000   4.0000   4.0000
3022  Default   46.000000    messag   46.000000   3.0000   3.0000
2545  Default   16.000000        bu   16.000000   2.0000   2.0000
2870  Default   31.000000      stop   31.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
2619  Topic10    5.312803     today   19.325094   1.6923  -5.4428
1337  Topic10    4.034422     drive   13.306399   1.7902  -5.7180
1849  Topic10    9.637205       abl   57.841355   1.1915  -4.8472
2759  Topic10    3.942926     becom   14.487001   1.6823  -5.7409
954   Topic10    6.406367      wait   37.079772   1.2278  -5.2556
117   Topic10   10.076083     phone   98.270734   0.7060  -4.8027
2295  Topic10    8.899419       see   84.745173   0.7299  -4.9269
2164  Topic10    6.098233     store   42.012270   1.0536  -5.3049
201   Topic10    6.084111     order   44.968395   0.9833  -5.3072
3205  Topic10    3.912846       job   16.900454   1.5205  -5.7486
601   Topic10    7.990017      make  115.999012   0.3082  -5.0347
2518  Topic10    7.441621      back   98.845262   0.3971  -5.1058
1813  Topic10    4.281500    ticket   25.035667   1.2176  -5.6586
852   Topic10   10.618419      time  292.746070  -0.3331  -4.7503
145   Topic10    7.112730      want  105.069699   0.2909  -5.1510
132   Topic10    8.933135      work  214.869935  -0.1967  -4.9231
1707  Topic10    5.617181   connect   56.172357   0.6810  -5.3870
1290  Top

/Users/liwenyu/anaconda/envs/my_env/lib/python2.7/site-packages/simplejson/encoder.py:262: DeprecationWarning: Interpreting naive datetime as local 2017-03-22 18:55:25.280533. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
